In [1]:
import os
from pyFIRS.wrappers import lastools

In [2]:
workdir = '/storage/lidar/odf_northwest_2015/abiqua'
raw = os.path.join(workdir, 'raw')
interim = '/storage/lidar/odf_northwest_2015/interim'

## Retile the data to add buffers for avoiding edge effects during processing.

In practice, executing the `lastile` command on individual tiles in parallel is likely to corrupt your output files. I suspect this is because the dynamic re-tiling of input files means that many output tiles are likely to require inputs from multiple input files, and that parallel processing outside of LAStools may result in collisions writing data from multiple inputs to these output tiles. So, for this case, we'll let `lastile` handle the parallelism under the hood. We won't have a progress bar, but this shouldn't take more than 5-10 minutes per ~100 tiles (with vendor tile size ~1000x1000m with 4-8 pts/m2).

**THERE ARE ARGUMENTS IN THE FOLLOWING COMMAND THAT DEPEND UPON THE UNITS OF THE DATA.**

In [3]:
las = lastools.useLAStools('/storage/lidar/LAStools/bin')

In [4]:
%%time
tile_proc = las.lastile(i=os.path.join(raw, '*.laz'),
                        tile_size=1000, # in units of lidar data
                        buffer=10, # assumes units are in meters
                        flag_as_withheld=True, # flag buffer points as "withheld", enables handling with other LAStools
                        extra_pass=True, # if outputting to LAZ format, can help avoid memory limits
                        full_bb=True,
                        olaz=True,
                        odir='/storage/lidar/odf_northwest_2015/interim/retiled',
                        cores=32);

CPU times: user 64 ms, sys: 20 ms, total: 84 ms
Wall time: 13min 18s
